In [2]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, random_split
import torch.optim as optim
from torchvision import transforms,datasets
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image


In [3]:
from create import create_dataloaders

In [4]:
img_list_path=r'C:\Users\ASUS RYZEN 7\Documents\PROYECTOS\Computer vision\animas\agri_data\images.txt'

In [5]:
import os
import shutil

# Ruta de la carpeta que contiene los archivos .txt
source_dir = r'C:\Users\ASUS RYZEN 7\Documents\PROYECTOS\Computer vision\animas\agri_data\images'

# Crear la carpeta 'labels' si no existe
labels_dir = os.path.join(source_dir, 'labels')
os.makedirs(labels_dir, exist_ok=True)

# Iterar sobre todos los archivos en el directorio
for filename in os.listdir(source_dir):
    if filename.endswith('.txt'):  # Si es un archivo .txt
        source_path = os.path.join(source_dir, filename)
        destination_path = os.path.join(labels_dir, filename)
        
        # Mover el archivo .txt a la carpeta 'labels'
        shutil.copy(source_path, destination_path)

print(f'Todos los archivos .txt se han movido a {labels_dir}')


Todos los archivos .txt se han movido a C:\Users\ASUS RYZEN 7\Documents\PROYECTOS\Computer vision\animas\agri_data\images\labels


In [6]:
import os

def create_images_txt(image_folder, output_txt):
    # Abrir el archivo para escribir
    with open(output_txt, "w") as file:
        # Recorrer todos los archivos en el folder de imágenes
        for filename in os.listdir(image_folder):
            # Si el archivo es una imagen (.jpg o .jpeg o .png)
            if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
                # Escribir la ruta completa de la imagen en el archivo
                img_path = os.path.join(image_folder, filename)
                file.write(f"{img_path}\n")

# Especifica la carpeta de imágenes y el archivo de salida
image_folder = r'C:\Users\ASUS RYZEN 7\Documents\PROYECTOS\Computer vision\animas\agri_data\images'
output_txt = r'C:\Users\ASUS RYZEN 7\Documents\PROYECTOS\Computer vision\animas\agri_data\images.txt'

create_images_txt(image_folder, output_txt)


In [7]:

# Abre tu imagen usando PIL
image = Image.open('image3.jpg')

In [8]:
class Conv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(Conv, self).__init__()
        # Definimos la convolución
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False)
        # Usamos ReLU como función de activación
        self.relu = nn.ReLU(inplace=True)
    
    def forward(self, x):
        # Aplicamos la convolución y luego la activación ReLU
        x = self.conv(x)
        x = self.relu(x)
        return x
# Definición de la clase Conv para una capa convolucional personalizada con LRN
class ConvLRN(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=1):
        super(ConvLRN, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.lrn = nn.LocalResponseNorm(size=5, alpha=1e-4, beta=0.75, k=2.0)  # LRN

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        x = self.lrn(x)  # Aplicar LRN después de la activación
        return x
    
class ConvBN(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, padding=0):
        super(ConvBN, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)  # Batch Normalization
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x
        
class MaxPooling(nn.Module):
    def __init__(self,kernel_size=3,stride=2,padding=0):
        super(MaxPooling,self).__init__()
        self.maxpool=nn.MaxPool2d(kernel_size=kernel_size,stride=stride,padding=padding)
        
    def forward(self,x):
        x=self.maxpool(x)
        return x


In [9]:
class YOLOV1(nn.Module):
    def __init__(self,S,B,num_classes):
        super(YOLOV1,self).__init__()
        self.S = S
        self.B = B
        self.num_classes = num_classes
        
        self.layer1=Conv(kernel_size=7,in_channels=3,out_channels=64,stride=2)
        self.maxpool1=MaxPooling(kernel_size=2,stride=2)
        
        self.layer2=Conv(kernel_size=3,in_channels=64,out_channels=192,stride=1)
        self.maxpool2=MaxPooling(kernel_size=2,stride=2)
        
        self.layer3=Conv(kernel_size=1,in_channels=192,out_channels=128,stride=1)
        self.layer4=Conv(kernel_size=3,in_channels=128,out_channels=256,stride=1)
        self.layer5=Conv(kernel_size=1,in_channels=256,out_channels=256,stride=1)
        self.layer6=Conv(kernel_size=3,in_channels=256,out_channels=512,stride=1)
        self.maxpool3=MaxPooling(kernel_size=2,stride=2)
        
        
        
        self.layer7=Conv(kernel_size=1,in_channels=512,out_channels=256,stride=1)
        self.layer8=Conv(kernel_size=3,in_channels=256,out_channels=512,stride=1)
        
        self.layer9=Conv(kernel_size=1,in_channels=512,out_channels=256,stride=1)
        self.layer10=Conv(kernel_size=3,in_channels=256,out_channels=512,stride=1)
        
        self.layer11=Conv(kernel_size=1,in_channels=512,out_channels=256,stride=1)
        self.layer12=Conv(kernel_size=3,in_channels=256,out_channels=512,stride=1)
        
        self.layer13=Conv(kernel_size=1,in_channels=512,out_channels=256,stride=1)
        self.layer14=Conv(kernel_size=3,in_channels=256,out_channels=512,stride=1)
        
        self.layer15=Conv(kernel_size=1,in_channels=512,out_channels=512,stride=1)
        self.layer16=Conv(kernel_size=3,in_channels=512,out_channels=1024,stride=1)
        
        
        
        self.maxpool4=MaxPooling(kernel_size=2,stride=2)
        
        
        
        self.layer17=Conv(kernel_size=1,in_channels=1024,out_channels=512,stride=1)
        self.layer18=Conv(kernel_size=3,in_channels=512,out_channels=1024,stride=1)
        
        self.layer19=Conv(kernel_size=1,in_channels=1024,out_channels=512,stride=1)
        self.layer20=Conv(kernel_size=3,in_channels=512,out_channels=1024,stride=1)
        
        self.layer21=Conv(kernel_size=3,in_channels=1024,out_channels=1024,stride=1)
        
        self.layer22=Conv(kernel_size=3,in_channels=1024,out_channels=1024,stride=2)
        
        self.layer23=Conv(kernel_size=3,in_channels=1024,out_channels=1024,stride=1)
        self.layer24=Conv(kernel_size=3,in_channels=1024,out_channels=1024,stride=1)
        
        self.fc1 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(1024 * 9* 9, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, self.S * self.S * (self.B * 5 + self.num_classes))  # Output de 7x7x30
        )
            
    def forward(self, x):
        x=self.layer1(x)
        x=self.maxpool1(x)
        
        x=self.layer2(x)
        x=self.maxpool2(x)
        
        
        x=self.layer3(x)
        x=self.layer4(x)
        x=self.layer5(x)
        x=self.layer6(x)
        x=self.maxpool3(x)
        
        x=self.layer7(x)
        x=self.layer8(x)
        
        x=self.layer9(x)
        x=self.layer10(x)
        
        x=self.layer11(x)
        x=self.layer12(x)
        
        x=self.layer13(x)
        x=self.layer14(x)
        
        x=self.layer15(x)
        x=self.layer16(x)
        x=self.maxpool4(x)
        
        x=self.layer17(x)
        x=self.layer18(x)
        
        x=self.layer19(x)
        x=self.layer20(x)
        
        x=self.layer21(x)
        x=self.layer22(x)
        
        x=self.layer23(x)
        x=self.layer24(x)
        
        # Aplana el tensor para que tenga el tamaño adecuado

        
        x=self.fc1(x)
        
          # Reestructuramos la salida a 7x7x30
        x = x.reshape(-1, self.S, self.S, self.B * 5 + self.num_classes)
        
        return x
        
        
            
            
            


In [10]:
import torch

def compute_iou(boxes1, boxes2):
    # Convertimos de (cx, cy, w, h) a (x1, y1, x2, y2)
    boxes1_t = torch.stack([boxes1[..., 0] - boxes1[..., 2] / 2.0,  # x1
                            boxes1[..., 1] - boxes1[..., 3] / 2.0,  # y1
                            boxes1[..., 0] + boxes1[..., 2] / 2.0,  # x2
                            boxes1[..., 1] + boxes1[..., 3] / 2.0], # y2
                           dim=-1)

    boxes2_t = torch.stack([boxes2[..., 0] - boxes2[..., 2] / 2.0,  # x1
                            boxes2[..., 1] - boxes2[..., 3] / 2.0,  # y1
                            boxes2[..., 0] + boxes2[..., 2] / 2.0,  # x2
                            boxes2[..., 1] + boxes2[..., 3] / 2.0], # y2
                           dim=-1)
    
    # Calculamos las coordenadas de la intersección
    lu = torch.max(boxes1_t[..., :2], boxes2_t[..., :2])  # Left Up (x1, y1)
    rd = torch.min(boxes1_t[..., 2:], boxes2_t[..., 2:])  # Right Down (x2, y2)

    # Calculamos el área de la intersección
    intersection = torch.clamp(rd - lu, min=0)
    inter_square = intersection[..., 0] * intersection[..., 1]

    # Área de las dos cajas
    square1 = boxes1[..., 2] * boxes1[..., 3]
    square2 = boxes2[..., 2] * boxes2[..., 3]

    # Calculamos la unión
    union_square = torch.clamp(square1 + square2 - inter_square, min=1e-10)

    # Calculamos el IOU
    iou = torch.clamp(inter_square / union_square, min=0.0, max=1.0)

    return iou


# Preparing dataset

In [11]:
#caso cuando sea estructura VOCpascal

train_images='ruta imagenes'
train_maps='ruta annotaciones xml'

#caso cuando sean txt ya listos


train_images=r'C:\Users\ASUS RYZEN 7\Documents\PROYECTOS\Computer vision\animas\agri_data\images'
train_maps=r'C:\Users\ASUS RYZEN 7\Documents\PROYECTOS\Computer vision\animas\agri_data\labels'

classes=['weed','crop']

B=2
N_CLASSES=len(classes)
H,W=224,224
OBJ_IND=N_CLASSES+5*B
SPLIT_SIZE=H/32
N_EPOCHS=3
LR=5e-4
BATCH_SIZE=16

In [12]:
import xml.etree.ElementTree as ET

In [13]:
def preprocess_xml(filename):
    tree= ET.parse(filename)
    root= tree.getroot()
    size_tree=root.find('size')
    height=float(size_tree.find('height').text)
    print(height)
    width=float(size_tree.find('width').text)
    print(width)
    bounding_boxes=[]
    for object_tree in root.findall('object'):
        for bounding_box in object_tree.iter('bnbdox'):
            xmin=(float(bounding_box.find('xmin').text))
            ymin=(float(bounding_box.find('ymin').text))
            xmax=(float(bounding_box.find('xmax').text))
            ymax=(float(bounding_box.find('ymax').text))
        class_name= object_tree.find('name').text
        class_dict={classes[i]: i for i in range(len(classes))}
        bounding_box=[(xmin+xmax)(2*width),(ymin+ymax)(2*height),(xmax-xmin)/width,(ymax-ymin)/height,class_dict[class_name]]
        bounding_boxes.append(bounding_box)
    
    return bounding_boxes

In [14]:
filename=r'C:\Users\ASUS RYZEN 7\Documents\PROYECTOS\Computer vision\animas\agri_data\labels\agri_0_5017.txt'
#suponiendo que vienen ya organizadas y normalizadas y con estrcutrua clase a,b,c,d
def preprocess_txt(filename):
    with open(filename, 'r') as file:
        contenido = file.read().strip()  # Leer el contenido y eliminar cualquier espacio en blanco extra
        contenido = contenido.split('\n')  # Dividir el contenido en una lista usando la coma como delimitador
    
    for i in range(0,len(contenido)):
        lista_aux=[]
        contenido[i]=contenido[i].split()
        # Verificar que la lista tenga al menos un elemento para evitar errores
    if len(contenido) < 1:
        raise ValueError("El archivo debe contener al menos un elemento.")
    
    for i in range(0,len(contenido)):
        contenido[i] = list(map(float, contenido[i] ))
        print(contenido[i])    
        # Mover el primer elemento al final de la lista
        primer_elemento = contenido[i].pop(0)  # Extraer el primer elemento
        contenido[i].append(primer_elemento)  # Añadir el primer elemento al final de la lista
    
    return contenido


In [15]:
preprocess_txt(filename)

[0.0, 0.485352, 0.632812, 0.611328, 0.542969]
[0.0, 0.635742, 0.331055, 0.587891, 0.349609]
[0.0, 0.182617, 0.148438, 0.361328, 0.292969]


[[0.485352, 0.632812, 0.611328, 0.542969, 0.0],
 [0.635742, 0.331055, 0.587891, 0.349609, 0.0],
 [0.182617, 0.148438, 0.361328, 0.292969, 0.0]]

In [16]:
import torch

In [17]:
import torch

def generate_output(bounding_boxes, length, split_size, n_classes):
    # Verifica que split_size y n_classes sean enteros
    split_size = int(split_size)
    n_classes = int(n_classes)
    
    # Inicializa el tensor con ceros
    output_label = torch.zeros((split_size, split_size, n_classes + 5*2), dtype=torch.float32)
    
    for b in range(length):
        grid_x = bounding_boxes[b][0] * split_size
        grid_y = bounding_boxes[b][1] * split_size
        i = int(grid_x)
        j = int(grid_y)
        
        if output_label[i, j, 0] == 0:
            # Usar indexación avanzada para actualizar el tensor
            output_label[i, j, 0:5] = torch.tensor([1., grid_x % 1, grid_y % 1, bounding_boxes[b][2], bounding_boxes[b][3]], dtype=torch.float32)
    
    return output_label



In [18]:
generate_output(preprocess_txt(filename),len(preprocess_txt(filename)),SPLIT_SIZE,N_CLASSES)[3][4]

[0.0, 0.485352, 0.632812, 0.611328, 0.542969]
[0.0, 0.635742, 0.331055, 0.587891, 0.349609]
[0.0, 0.182617, 0.148438, 0.361328, 0.292969]
[0.0, 0.485352, 0.632812, 0.611328, 0.542969]
[0.0, 0.635742, 0.331055, 0.587891, 0.349609]
[0.0, 0.182617, 0.148438, 0.361328, 0.292969]


tensor([1.0000, 0.3975, 0.4297, 0.6113, 0.5430, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000])

In [19]:
from utils.transforms import RandomHorizontalFlip, RandomVerticalFlip, RandomRotationJitter, RandomBlur
from datasets import YoloDataset

In [20]:
import os
import torch
from PIL import Image
from torch.utils.data import Dataset
from utils import encode

class YoloDataset(Dataset):
    def __init__(self, img_list_path, S, B, num_classes, transforms=None, img_box_transforms=None, eval_mode=False):
        with open(img_list_path, "r") as img_list_file:
            self.img_filenames = img_list_file.readlines()
            
        self.img_filenames = list(map(lambda x:x.strip(), self.img_filenames))
        self.label_files = []
        for path in self.img_filenames:
            image_dir = os.path.dirname(path)
            label_dir = "labels".join(image_dir.rsplit("images", 1))
            assert label_dir != image_dir, \
                f"Image path must contain a folder named 'images'! \n'{image_dir}'"
            label_file = os.path.join(label_dir, os.path.basename(path))
            label_file = os.path.splitext(label_file)[0] + '.txt'
            self.label_files.append(label_file)

        self.transforms = transforms
        self.img_box_transforms = img_box_transforms
        
        self.S = S
        self.B = B
        self.num_classes = num_classes
        self.eval_mode = eval_mode

    def eval(self, eval_mode=True):
        self.eval_mode = eval_mode
        return

    def __len__(self):
        return len(self.img_filenames)

    def __getitem__(self, idx):
        # read image
        img_filename = self.img_filenames[idx]
        img = Image.open(img_filename, mode='r')
        if self.transforms is not None:
            img = self.transforms(img)

        # read each image's corresponding label (.txt)
        labels = []
        with open(self.label_files[idx], 'r') as f:
            lines = f.readlines()
            for line in lines:
                if line == '\n':
                    continue
                line = line.strip().split(' ')
                # convert xywh str to float, class str to int
                c, x, y, w, h = int(line[0]), float(line[1]), float(line[2]), float(line[3]), float(line[4])
                if self.eval_mode: 
                    labels.append((x, y, w, h, 1.0, c))
                else:
                    labels.append((x, y, w, h, c))
                
        if self.img_box_transforms is not None:
            for t in self.img_box_transforms:
                img, labels = t(img, labels)
        
        if self.eval_mode: 
            return img, torch.Tensor(labels)

        encoded_labels = encode(labels, self.S, self.B, self.num_classes)  # convert label list to encoded label
        encoded_labels = torch.Tensor(encoded_labels)
        return img, encoded_labels


In [21]:
input_size=224
transform = transforms.Compose([
        transforms.Resize((input_size, input_size)),
        transforms.ColorJitter(0.2, 0.5, 0.7, 0.07),
        transforms.RandomAdjustSharpness(3, p=0.2),
        RandomBlur(kernel_size=[3,3], sigma=[0.1, 2], p=0.1),
        transforms.RandomGrayscale(p=0.1),
        transforms.ToTensor()
    ])
img_box_transform = [
        RandomHorizontalFlip(0.5),
        RandomVerticalFlip(0.05),
        RandomRotationJitter()
    ]

    # create yolo dataset
dataset1= YoloDataset(img_list_path, 7, B, 2, transforms=transform, img_box_transforms=img_box_transform)

In [22]:
dataset1.__getitem__(1)[1].shape

torch.Size([7, 7, 12])

In [23]:
from PIL import Image
def get_imbboxes(im_path,txt_path):
    img=img = Image.open(im_path, mode='r')
    bboxes=preprocess_txt(txt_path)
    return img,bboxes



In [24]:
img_list_path
with open(img_list_path, 'r') as file:
        contenido = file.read().strip()  # Leer el contenido y eliminar cualquier espacio en blanco extra
        contenido = contenido.split('\n')

In [25]:
nombre_archivos=[]
for i in range(len(contenido)):
    nombre_archivos.append(contenido[i].split('\\')[-1].split('.')[0])
nombre_archivos
tuplas_im_box=[]
base_path=r'C:\Users\ASUS RYZEN 7\Documents\PROYECTOS\Computer vision\animas\agri_data'
for i in range(len(nombre_archivos)):
    tuplas_im_box.append((base_path+r'\images'+'\\'+nombre_archivos[i],base_path+r'\labels'+'\\'+nombre_archivos[i]))




In [26]:
tuplas_im_box

[('C:\\Users\\ASUS RYZEN 7\\Documents\\PROYECTOS\\Computer vision\\animas\\agri_data\\images\\agri_0_1009',
  'C:\\Users\\ASUS RYZEN 7\\Documents\\PROYECTOS\\Computer vision\\animas\\agri_data\\labels\\agri_0_1009'),
 ('C:\\Users\\ASUS RYZEN 7\\Documents\\PROYECTOS\\Computer vision\\animas\\agri_data\\images\\agri_0_1017',
  'C:\\Users\\ASUS RYZEN 7\\Documents\\PROYECTOS\\Computer vision\\animas\\agri_data\\labels\\agri_0_1017'),
 ('C:\\Users\\ASUS RYZEN 7\\Documents\\PROYECTOS\\Computer vision\\animas\\agri_data\\images\\agri_0_1018',
  'C:\\Users\\ASUS RYZEN 7\\Documents\\PROYECTOS\\Computer vision\\animas\\agri_data\\labels\\agri_0_1018'),
 ('C:\\Users\\ASUS RYZEN 7\\Documents\\PROYECTOS\\Computer vision\\animas\\agri_data\\images\\agri_0_1020',
  'C:\\Users\\ASUS RYZEN 7\\Documents\\PROYECTOS\\Computer vision\\animas\\agri_data\\labels\\agri_0_1020'),
 ('C:\\Users\\ASUS RYZEN 7\\Documents\\PROYECTOS\\Computer vision\\animas\\agri_data\\images\\agri_0_1024',
  'C:\\Users\\ASUS RYZEN

In [27]:
import os
import torch
from PIL import Image
from torch.utils.data import Dataset

def preprocess_txt(filename):
    with open(filename, 'r') as file:
        contenido = file.read().strip()
        contenido = contenido.split('\n')
    
    bounding_boxes = []
    for line in contenido:
        if line.strip():
            parts = line.split()
            class_id = int(parts[0])
            x_center = float(parts[1])
            y_center = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])
            bounding_boxes.append((x_center, y_center, width, height, class_id))
    
    return bounding_boxes

import torch

def generate_output(bounding_boxes, length, split_size, n_classes, B=2):
    split_size = int(split_size)
    n_classes = int(n_classes)
    
    # El tensor de salida tiene dimensiones [split_size, split_size, B*5 + n_classes]
    output_label = torch.zeros((split_size, split_size, B * 5 + n_classes), dtype=torch.float32)
    
    box_index = 0  # Índice para cada caja
    for b in range(length):
        grid_x = bounding_boxes[b][0] * split_size
        grid_y = bounding_boxes[b][1] * split_size
        i = int(grid_x)
        j = int(grid_y)

        # Encontramos el primer espacio disponible para la caja
        for box in range(B):
            if output_label[i, j, box * 5] == 0:  # Si no se ha asignado ninguna caja en este lugar
                # Asignamos los valores de la caja: confianza, x, y, ancho, alto
                output_label[i, j, box * 5:(box + 1) * 5] = torch.tensor(
                    [1., grid_x % 1, grid_y % 1, bounding_boxes[b][2], bounding_boxes[b][3]],
                    dtype=torch.float32
                )
                
                # Asignamos las clases (inicialmente 0s con un 1 en la clase correspondiente)
                class_id = bounding_boxes[b][-1]
                output_label[i, j, B * 5 + class_id] = 1.0
                break
    
    return output_label


class YoloDatasetOwn(Dataset):
    def __init__(self, img_list_path, S, B, num_classes, transforms=None, img_box_transforms=None):
        with open(img_list_path, "r") as img_list_file:
            self.img_filenames = img_list_file.readlines()
        
        self.img_filenames = list(map(lambda x: x.strip(), self.img_filenames))
        self.label_files = []
        for path in self.img_filenames:
            image_dir = os.path.dirname(path)
            label_dir = "labels".join(image_dir.rsplit("images", 1))
            assert label_dir != image_dir, \
                f"Image path must contain a folder named 'images'! \n'{image_dir}'"
            label_file = os.path.join(label_dir, os.path.basename(path))
            label_file = os.path.splitext(label_file)[0] + '.txt'
            self.label_files.append(label_file)

        self.transforms = transforms
        self.img_box_transforms = img_box_transforms
        
        self.S = S
        self.B = B
        self.num_classes = num_classes

    def __len__(self):
        return len(self.img_filenames)

    def __getitem__(self, idx):
        # Read image
        img_filename = self.img_filenames[idx]
        img = Image.open(img_filename).convert("RGB")
        
        # Get image dimensions before transformations
        width, height = img.size
        
        if self.transforms is not None:
            img = self.transforms(img)
        
        # Read labels
        label_file = self.label_files[idx]
        bounding_boxes = preprocess_txt(label_file)
        
        # Normalize bounding boxes
        bounding_boxes = [(x / width, y / height, w / width, h / height, c) for (x, y, w, h, c) in bounding_boxes]
        
        # Generate YOLO output
        encoded_labels = generate_output(bounding_boxes, len(bounding_boxes), self.S, self.num_classes)
        
        return img, encoded_labels





In [28]:
class YoloDatasetOwn(Dataset):
    def __init__(self, img_list_path, S, B, num_classes, transforms=None, img_box_transforms=None):
        with open(img_list_path, "r") as img_list_file:
            self.img_filenames = img_list_file.readlines()
        
        self.img_filenames = list(map(lambda x: x.strip(), self.img_filenames))
        self.label_files = []
        for path in self.img_filenames:
            image_dir = os.path.dirname(path)
            label_dir = "labels".join(image_dir.rsplit("images", 1))
            assert label_dir != image_dir, \
                f"Image path must contain una carpeta llamada 'images'! \n'{image_dir}'"
            label_file = os.path.join(label_dir, os.path.basename(path))
            label_file = os.path.splitext(label_file)[0] + '.txt'
            self.label_files.append(label_file)

        self.transforms = transforms
        self.img_box_transforms = img_box_transforms
        
        self.S = S
        self.B = B
        self.num_classes = num_classes

    def __len__(self):
        return len(self.img_filenames)

    def __getitem__(self, idx):
        # Read image
        img_filename = self.img_filenames[idx]
        img = Image.open(img_filename).convert("RGB")
        
        # Get image dimensions before transformations
        width, height = img.size
        
        if self.transforms is not None:
            img = self.transforms(img)
        
        # Read labels
        label_file = self.label_files[idx]
        bounding_boxes = preprocess_txt(label_file)
        
        # Normalize bounding boxes (x_center, y_center, width, height)
        bounding_boxes = [(x / width, y / height, w / width, h / height, c) for (x, y, w, h, c) in bounding_boxes]
        
        # Generate YOLO output (encoded_labels)
        encoded_labels = generate_output(bounding_boxes, len(bounding_boxes), self.S, self.num_classes, B=self.B)
        
        return img, encoded_labels


In [29]:
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((227, 227)),  # Ajusta el tamaño según sea necesario
    transforms.ToTensor(),
])

dataset2 = YoloDatasetOwn(
    img_list_path=r'C:\Users\ASUS RYZEN 7\Documents\PROYECTOS\Computer vision\animas\agri_data\images.txt',
    S=7,
    B=2,
    num_classes=2,
    transforms=transform
)



In [30]:
dataset2.__getitem__(0)[1].shape

torch.Size([7, 7, 12])

In [32]:
dataloader = DataLoader(dataset2, batch_size=16, shuffle=True,num_workers=0)


In [35]:
dataloader1,val=create_dataloaders(img_list_path, 0.8, 0.2, 16, input_size, 7, B, 2)

In [43]:
next(iter(dataloader1))[1].shape

torch.Size([16, 7, 7, 12])

In [42]:
next(iter(dataloader))[1].shape

torch.Size([16, 7, 7, 12])

# YOLO LOSS

In [44]:
import numpy as np
import torch
from collections import Counter

def encode(bboxes, S, B, num_classes):
    # bbox is cx, cy, width, height, classification
    label = np.zeros((S, S, 5 * B + num_classes))
    for x, y, w, h, c in bboxes:
        x_grid = int(x // (1.0 / S))
        y_grid = int(y // (1.0 / S))
        xx, yy = x, y
        label[y_grid, x_grid, 0:5] = np.array([xx, yy, w, h, 1])
        label[y_grid, x_grid, 5:10] = np.array([xx, yy, w, h, 1])
        label[y_grid, x_grid, 10 + c] = 1
    return label


def decode(pred, S, B, num_classes, conf_thresh, iou_thresh):
    bboxes = torch.zeros((S * S, 5 + num_classes))  # 49*25
    for x in range(S):
        for y in range(S):
            conf1, conf2 = pred[x, y, 4], pred[x, y, 9]
            if conf1 > conf2:
                # bbox1
                bboxes[(x * S + y), 0:4] = torch.Tensor([pred[x, y, 0], pred[x, y, 1], pred[x, y, 2], pred[x, y, 3]])
                bboxes[(x * S + y), 4] = pred[x, y, 4]
                bboxes[(x * S + y), 5:] = pred[x, y, 10:]
            else:
                # bbox2
                bboxes[(x * S + y), 0:4] = torch.Tensor([pred[x, y, 5], pred[x, y, 6], pred[x, y, 7], pred[x, y, 8]])
                bboxes[(x * S + y), 4] = pred[x, y, 9]
                bboxes[(x * S + y), 5:] = pred[x, y, 10:]

    # apply NMS to all bounding boxes
    decoded_bbox = nms(bboxes, num_classes, conf_thresh, iou_thresh)
    return decoded_bbox # x y w h c

def iou(bbox1, bbox2):
    # bbox: x y w h
    bbox1, bbox2 = bbox1.cpu().detach().numpy().tolist(), bbox2.cpu().detach().numpy().tolist()

    area1 = bbox1[2] * bbox1[3]  # bbox1's area
    area2 = bbox2[2] * bbox2[3]  # bbox2's area

    max_left = max(bbox1[0] - bbox1[2] / 2, bbox2[0] - bbox2[2] / 2)
    min_right = min(bbox1[0] + bbox1[2] / 2, bbox2[0] + bbox2[2] / 2)
    max_top = max(bbox1[1] - bbox1[3] / 2, bbox2[1] - bbox2[3] / 2)
    min_bottom = min(bbox1[1] + bbox1[3] / 2, bbox2[1] + bbox2[3] / 2)

    if max_left >= min_right or max_top >= min_bottom:
        return 0
    else:
        # iou = intersect / union
        intersect = (min_right - max_left) * (min_bottom - max_top)
        return intersect / (area1 + area2 - intersect)

def nms(bboxes, num_classes, conf_thresh=0.1, iou_thresh=0.3):
    # Non-Maximum Suppression
    bbox_prob = bboxes[:, 5:].clone().detach()
    bbox_conf = bboxes[:, 4].clone().detach().unsqueeze(1).expand_as(bbox_prob)
    class_conf = bbox_conf * bbox_prob 
    class_conf[class_conf <= conf_thresh] = 0

    # for each class, sort the class confidence
    for c in range(num_classes):
        rank = torch.sort(class_conf[:, c], descending=True).indices 
        # for each bbox
        for i in range(bboxes.shape[0]):
            if class_conf[rank[i], c] == 0:
                continue
            for j in range(i + 1, bboxes.shape[0]):
                if class_conf[rank[j], c] != 0:
                    curr_iou = iou(bboxes[rank[i], 0:4], bboxes[rank[j], 0:4])
                    if curr_iou > iou_thresh:
                        class_conf[rank[j], c] = 0

    # exclude class confidence score equals to 0
    bboxes = bboxes[torch.max(class_conf, dim=1).values > 0]

    class_conf = class_conf[torch.max(class_conf, dim=1).values > 0]

    ret = torch.ones((bboxes.size()[0], 6))

    # return null
    if bboxes.size()[0] == 0:
        return torch.tensor([])

    # bbox coord
    ret[:, 0:4] = bboxes[:, 0:4]
    # bbox class confidence
    ret[:, 4] = torch.max(class_conf, dim=1).values
    # bbox class
    ret[:, 5] = torch.argmax(bboxes[:, 5:], dim=1).int()
    return ret



In [45]:
class YoloLoss(nn.Module):
    def __init__(self, S, B):
        super().__init__()
        self.S = S
        self.B = B

    def forward(self, preds, targets):
        batch_size = targets.size(0)

        coord_xy_loss = 0.  # coord xy loss
        coord_wh_loss = 0.  # coord wh loss
        obj_loss = 0.  # obj loss
        no_obj_loss = 0.  # no obj loss
        class_loss = 0.  # class loss

        for i in range(batch_size):
            for y in range(self.S):
                for x in range(self.S):
                    # this region has object
                    if targets[i, y, x, 4] == 1:
                        pred_bbox1 = torch.Tensor(
                            [preds[i, y, x, 0], preds[i, y, x, 1], preds[i, y, x, 2], preds[i, y, x, 3]])
                        pred_bbox2 = torch.Tensor(
                            [preds[i, y, x, 5], preds[i, y, x, 6], preds[i, y, x, 7], preds[i, y, x, 8]])
                        target_bbox = torch.Tensor(
                            [targets[i, y, x, 0], targets[i, y, x, 1], targets[i, y, x, 2], targets[i, y, x, 3]])

                        # compute iou of two bbox
                        iou1 = iou(pred_bbox1, target_bbox)
                        iou2 = iou(pred_bbox2, target_bbox)

                        # judge responsible box
                        if iou1 > iou2:
                            # calculate coord xy loss
                            coord_xy_loss += 5 * torch.sum((targets[i, y, x, 0:2] - preds[i, y, x, 0:2]) ** 2)

                            # coord wh loss
                            coord_wh_loss += torch.sum((targets[i, y, x, 2:4].sqrt() - preds[i, y, x, 2:4].sqrt()) ** 2)

                            # obj confidence loss
                            obj_loss += (iou1 - preds[i, y, x, 4]) ** 2

                            # no obj confidence loss
                            no_obj_loss += 0.5 * ((0 - preds[i, y, x, 9]) ** 2)
                        else:
                            # coord xy loss
                            coord_xy_loss += 5 * torch.sum((targets[i, y, x, 5:7] - preds[i, y, x, 5:7]) ** 2)

                            # coord wh loss
                            coord_wh_loss += torch.sum((targets[i, y, x, 7:9].sqrt() - preds[i, y, x, 7:9].sqrt()) ** 2)

                            # obj confidence loss
                            obj_loss += (iou2 - preds[i, y, x, 9]) ** 2

                            # no obj confidence loss
                            no_obj_loss += 0.5 * ((0 - preds[i, y, x, 4]) ** 2)

                        # class loss
                        class_loss += torch.sum((targets[i, y, x, 10:] - preds[i, y, x, 10:]) ** 2)

                    # this region has no object
                    else:
                        no_obj_loss += 0.5 * torch.sum((0 - preds[i, y, x, [4, 9]]) ** 2)

        return (coord_xy_loss + coord_wh_loss + obj_loss + no_obj_loss + class_loss) / batch_size   # loss 


In [46]:
import torch
import torch.nn as nn

# Definir dimensiones
S = 7  # Tamaño de la cuadrícula
B = 2  # Número de cajas predichas por celda
C = 20  # Número de clases
batch_size = 16

# Crear un ejemplo de predicciones y etiquetas
preds = torch.rand((batch_size, S, S, B * 5 + C))  # Predicciones aleatorias
targets = torch.zeros((batch_size, S, S, B * 5 + C))  # Etiquetas simuladas

# Definir una etiqueta verdadera para una celda
targets[0, 2, 3, 4] = 1  # Indicar que hay un objeto en la celda [2, 3]
targets[0, 2, 3, :4] = torch.tensor([0.5, 0.5, 1.0, 1.0])  # Coordenadas y tamaño de la caja
targets[0, 2, 3, 10] = 1  # Clase asociada al objeto

# Crear instancia de la clase de pérdida
loss_fn = YoloLoss(S=S, B=B)

# Calcular la pérdida
loss = loss_fn(preds, targets)

# Imprimir la pérdida
print(f"YOLO Loss: {loss.item()}")


YOLO Loss: 17.149045944213867


In [47]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import OneCycleLR
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image


In [48]:
S = 7
B = 2
num_classes = 2
batch_size = 2
num_epochs = 10
learning_rate = 1e-3

In [49]:
# Initialize model, loss function, and optimizer
model = YOLOV1(S, B, num_classes)
criterion = YoloLoss(S, B)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = OneCycleLR(optimizer, max_lr=learning_rate, steps_per_epoch=len(dataloader), epochs=num_epochs)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, targets in tqdm(dataloader):
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        running_loss += loss.item() * images.size(0)
    
    epoch_loss = running_loss / len(dataset2)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# Save the trained model
torch.save(model.state_dict(), 'yolov1.pth')

# Plotting dummy data for demonstration
plt.plot([i for i in range(num_epochs)], [0.1 * (num_epochs - i) for i in range(num_epochs)], label="Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Training Loss Over Epochs")
plt.legend()
plt.show()


  2%|▏         | 2/82 [03:15<2:10:21, 97.77s/it]


KeyboardInterrupt: 

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from torchvision.transforms import functional as F

# Load the trained model
S = 7
B = 2
num_classes = 20
model = YOLOV1(S, B, num_classes)
model.load_state_dict(torch.load('yolov1.pth'))
model.eval()

def draw_boxes(image, boxes, class_ids, class_names):
    """Draw bounding boxes on the image."""
    draw = ImageDraw.Draw(image)
    for i, (box, class_id) in enumerate(zip(boxes, class_ids)):
        x_center, y_center, width, height = box
        x1 = int((x_center - width / 2) * image.width)
        y1 = int((y_center - height / 2) * image.height)
        x2 = int((x_center + width / 2) * image.width)
        y2 = int((y_center + height / 2) * image.height)
        draw.rectangle([x1, y1, x2, y2], outline="red", width=2)
        draw.text((x1, y1), f"{class_names[class_id]}", fill="red")
    return image

def evaluate(model, image_paths, class_names):
    """Evaluate the model and display images with bounding boxes."""
    for image_path in image_paths:
        image = Image.open(image_path).convert("RGB")
        image_tensor = F.to_tensor(image).unsqueeze(0)
        
        with torch.no_grad():
            outputs = model(image_tensor)
        
        # Process the outputs to get bounding boxes and class IDs
        # Note: Adjust this processing based on your model's output format
        boxes = []  # Replace with actual box extraction logic
        class_ids = []  # Replace with actual class ID extraction logic
        
        # Draw bounding boxes
        image_with_boxes = draw_boxes(image, boxes, class_ids, class_names)
        
        # Display the image
        plt.imshow(image_with_boxes)
        plt.axis('off')
        plt.show()

# Example usage
class_names = ['class1', 'class2', 'class3']  # Replace with your actual class names
image_paths = ['path/to/image1.jpg', 'path/to/image2.jpg']  # Replace with your actual image paths
evaluate(model, image_paths, class_names)